In [1]:
!pip install torch torchvision numpy tqdm



import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
import numpy as np

class CustomFashionMNIST(Dataset):
    def _init_(self, root='./data', train=True, transform=None):
        self.original_dataset = datasets.FashionMNIST(
            root=root,
            train=train,
            download=True
        )

        self.data = self.original_dataset.data.numpy()
        self.targets = self.original_dataset.targets.numpy()
        self.data = self.data.astype(np.float32) / 255.0
        self.transform = transform

        self.classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                       'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    def _len_(self):
        return len(self.data)

    def _getitem_(self, idx):
        image = self.data[idx]
        label = self.targets[idx]
        image = torch.FloatTensor(image).unsqueeze(0)

        if self.transform:
            image = self.transform(image)

        return image, label

class FashionNet(nn.Module):
    def _init_(self):
        super(FashionNet, self)._init_()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)

        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = F.relu(self.bn3(self.conv3(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

def save_model(model, optimizer, epoch, accuracy, is_best=False):
    """Save model weights - both best and last"""
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'accuracy': accuracy
    }

    # Always save the last model
    last_filename = 'fashion_model_last.pt'
    torch.save(checkpoint, last_filename)  # Removed weights_only parameter
    print(f"Last model saved to {last_filename}")

    # Save best model separately if it's the best
    if is_best:
        best_filename = 'fashion_model_best.pt'
        torch.save(checkpoint, best_filename)  # Removed weights_only parameter
        print(f"Best model saved to {best_filename}")

def load_model(filename='fashion_model_best.pt'):
    """Load model weights"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = FashionNet().to(device)

    try:
        checkpoint = torch.load(filename, map_location=device)  # Removed weights_only parameter
        model.load_state_dict(checkpoint['model_state_dict'])
        accuracy = checkpoint.get('accuracy', 0.0)
        epoch = checkpoint.get('epoch', 0)
        print(f"Model loaded from {filename}")
        print(f"Accuracy: {accuracy:.2f}%")
        return model, accuracy, epoch
    except FileNotFoundError:
        print(f"No saved model found at {filename}")
        return model, 0.0, 0

if _name_ == "_main_":
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Create datasets and loaders
    train_dataset = CustomFashionMNIST(train=True)
    test_dataset = CustomFashionMNIST(train=False)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    # Create model
    model = FashionNet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # Train for 100 epochs
    num_epochs = 100
    best_accuracy = 0.0

    print("Starting training...")

    for epoch in range(num_epochs):
        # Training
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}')

        # Evaluation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        accuracy = 100 * correct / total
        print(f'Epoch: {epoch}, Accuracy: {accuracy:.2f}%')

        # Save both best and last models
        is_best = accuracy > best_accuracy
        if is_best:
            best_accuracy = accuracy
            print(f'New best accuracy: {accuracy:.2f}%')
        save_model(model, optimizer, epoch, accuracy, is_best=is_best)

    print(f"\nTraining completed!")
    print(f"Best accuracy achieved: {best_accuracy:.2f}%")

NameError: name '_name_' is not defined

In [3]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
import numpy as np

class CustomFashionMNIST(Dataset):
    def _init_(self, root='./data', train=True, transform=None):
        self.original_dataset = datasets.FashionMNIST(
            root=root,
            train=train,
            download=True
        )

        self.data = self.original_dataset.data.numpy()
        self.targets = self.original_dataset.targets.numpy()
        self.data = self.data.astype(np.float32) / 255.0
        self.transform = transform

        self.classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                       'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    def _len_(self):
        return len(self.data)

    def _getitem_(self, idx):
        image = self.data[idx]
        label = self.targets[idx]
        image = torch.FloatTensor(image).unsqueeze(0)

        if self.transform:
            image = self.transform(image)

        return image, label

class FashionNet(nn.Module):
    def _init_(self):
        super(FashionNet, self)._init_()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)

        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = F.relu(self.bn3(self.conv3(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

def save_model(model, optimizer, epoch, accuracy, is_best=False):

    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'accuracy': accuracy
    }


    last_filename = 'fashion_model_last.pt'
    torch.save(checkpoint, last_filename)
    print(f"Last model saved to {last_filename}")


    if is_best:
        best_filename = 'fashion_model_best.pt'
        torch.save(checkpoint, best_filename)
        print(f"Best model saved to {best_filename}")

def load_model(filename='fashion_model_best.pt'):
    """Load model weights"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = FashionNet().to(device)

    try:
        checkpoint = torch.load(filename, map_location=device)  # Removed weights_only parameter
        model.load_state_dict(checkpoint['model_state_dict'])
        accuracy = checkpoint.get('accuracy', 0.0)
        epoch = checkpoint.get('epoch', 0)
        print(f"Model loaded from {filename}")
        print(f"Accuracy: {accuracy:.2f}%")
        return model, accuracy, epoch
    except FileNotFoundError:
        print(f"No saved model found at {filename}")
        return model, 0.0, 0

if _name_ == "_main_":
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Create datasets and loaders
    train_dataset = CustomFashionMNIST(train=True)
    test_dataset = CustomFashionMNIST(train=False)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    # Create model
    model = FashionNet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # Train for 100 epochs
    num_epochs = 100
    best_accuracy = 0.0

    print("Starting training...")

    for epoch in range(num_epochs):
        # Training
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}')

        # Evaluation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        accuracy = 100 * correct / total
        print(f'Epoch: {epoch}, Accuracy: {accuracy:.2f}%')

        # Save both best and last models
        is_best = accuracy > best_accuracy
        if is_best:
            best_accuracy = accuracy
            print(f'New best accuracy: {accuracy:.2f}%')
        save_model(model, optimizer, epoch, accuracy, is_best=is_best)

    print(f"\nTraining completed!")
    print(f"Best accuracy achieved: {best_accuracy:.2f}%")

NameError: name '_name_' is not defined

In [8]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
import numpy as np

class CustomFashionMNIST(Dataset):
    def _init_(self, root='./data', train=True, transform=None):
        self.original_dataset = datasets.FashionMNIST(
            root=root,
            train=train,
            download=True
        )

        self.data = self.original_dataset.data.numpy()
        self.targets = self.original_dataset.targets.numpy()
        self.data = self.data.astype(np.float32) / 255.0
        self.transform = transform

        self.classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                       'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    def _len_(self):
        return len(self.data)

    def _getitem_(self, idx):
        image = self.data[idx]
        label = self.targets[idx]
        image = torch.FloatTensor(image).unsqueeze(0)

        if self.transform:
            image = self.transform(image)

        return image, label

class FashionNet(nn.Module):
    def _init_(self):
        super(FashionNet, self)._init_()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)

        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = F.relu(self.bn3(self.conv3(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

def save_model(model, optimizer, epoch, accuracy, is_best=False):

    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'accuracy': accuracy
    }


    last_filename = 'fashion_model_last.pt'
    torch.save(checkpoint, last_filename)
    print(f"Last model saved to {last_filename}")


    if is_best:
        best_filename = 'fashion_model_best.pt'
        torch.save(checkpoint, best_filename)
        print(f"Best model saved to {best_filename}")

def load_model(filename='fashion_model_best.pt'):
    """Load model weights"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = FashionNet().to(device)

    try:
        checkpoint = torch.load(filename, map_location=device)  # Removed weights_only parameter
        model.load_state_dict(checkpoint['model_state_dict'])
        accuracy = checkpoint.get('accuracy', 0.0)
        epoch = checkpoint.get('epoch', 0)
        print(f"Model loaded from {filename}")
        print(f"Accuracy: {accuracy:.2f}%")
        return model, accuracy, epoch
    except FileNotFoundError:
        print(f"No saved model found at {filename}")
        return model, 0.0, 0

if _name_ == "_main_":
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Create datasets and loaders
    train_dataset = CustomFashionMNIST(train=True)
    test_dataset = CustomFashionMNIST(train=False)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    # Create model
    model = FashionNet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # Train for 35 epochs
    num_epochs = 35
    best_accuracy = 0.0

    print("Starting training...")

    for epoch in range(num_epochs):
        # Training
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}')

        # Evaluation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        accuracy = 100 * correct / total
        print(f'Epoch: {epoch}, Accuracy: {accuracy:.2f}%')

        # Save both best and last models
        is_best = accuracy > best_accuracy
        if is_best:
            best_accuracy = accuracy
            print(f'New best accuracy: {accuracy:.2f}%')
        save_model(model, optimizer, epoch, accuracy, is_best=is_best)

    print(f"\nTraining completed!")
    print(f"Best accuracy achieved: {best_accuracy:.2f}%")

NameError: name '_name_' is not defined

In [10]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
import numpy as np

class ImageDataset(Dataset):
    def __init__(self, path='./data', is_train=True, transform=None):
        self.dataset = datasets.FashionMNIST(
            root=path,
            train=is_train,
            download=True
        )

        self.images = self.dataset.data.numpy()
        self.labels = self.dataset.targets.numpy()
        self.images = self.images.astype(np.float32) / 255.0
        self.transform = transform

        self.class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                            'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        image = torch.FloatTensor(image).unsqueeze(0)

        if self.transform:
            image = self.transform(image)

        return image, label

class ConvolutionalNet(nn.Module):
    def __init__(self):
        super(ConvolutionalNet, self).__init__()
        # Defining layers
        self.conv_layer1 = nn.Conv2d(1, 32, 3, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(32)
        self.conv_layer2 = nn.Conv2d(32, 64, 3, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(64)
        self.conv_layer3 = nn.Conv2d(64, 64, 3, padding=1)
        self.batch_norm3 = nn.BatchNorm2d(64)

        self.pooling_layer = nn.MaxPool2d(2, 2)
        self.fc_layer1 = nn.Linear(64 * 7 * 7, 512)
        self.dropout_layer = nn.Dropout(0.5)
        self.fc_layer2 = nn.Linear(512, 10)

    def forward(self, x):

        x = self.pooling_layer(F.relu(self.batch_norm1(self.conv_layer1(x))))
        x = self.pooling_layer(F.relu(self.batch_norm2(self.conv_layer2(x))))
        x = F.relu(self.batch_norm3(self.conv_layer3(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc_layer1(x))
        x = self.dropout_layer(x)
        x = self.fc_layer2(x)
        return x

def save_best_model(model, filepath='best_model.pt'):

    torch.save(model.state_dict(), filepath)
    print(f"Best model saved as {filepath}")

def load_best_model(filepath='best_model.pt'):
    """Load the best model's weights"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = ConvolutionalNet().to(device)

    try:
        model.load_state_dict(torch.load(filepath, map_location=device))
        print(f"Model loaded from {filepath}")
        return model
    except FileNotFoundError:
        print(f"Model file {filepath} not found.")
        return model

if __name__ == "__main__":

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Training started on device: {device}")
    train_dataset = ImageDataset(is_train=True)
    test_dataset = ImageDataset(is_train=False)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    model = ConvolutionalNet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_function = nn.CrossEntropyLoss()
    total_epochs = 85
    highest_accuracy = 0.0
    best_model_path = 'best_model.pt'

    print("Training process initiated...")

    for epoch in range(total_epochs):

        model.train()
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            predictions = model(inputs)
            loss = loss_function(predictions, targets)
            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print(f"Epoch {epoch}, Batch {batch_idx}: Current Loss = {loss.item():.4f}")

        model.eval()
        correct_predictions = 0
        total_samples = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                predictions = model(inputs)
                _, predicted_labels = torch.max(predictions.data, 1)
                total_samples += targets.size(0)
                correct_predictions += (predicted_labels == targets).sum().item()

        accuracy = 100 * correct_predictions / total_samples
        print(f"Epoch {epoch}, Test Accuracy: {accuracy:.2f}%")

        if accuracy > highest_accuracy:
            highest_accuracy = accuracy
            print(f"New best accuracy: {accuracy:.2f}%")
            save_best_model(model, best_model_path)

    print(f"\nTraining complete!")
    print(f"Highest accuracy achieved during training: {highest_accuracy:.2f}%")


Training started on device: cuda
Training process initiated...
Epoch 0, Batch 0: Current Loss = 2.3439
Epoch 0, Batch 100: Current Loss = 0.3413
Epoch 0, Batch 200: Current Loss = 0.3487
Epoch 0, Batch 300: Current Loss = 0.4281
Epoch 0, Batch 400: Current Loss = 0.2316
Epoch 0, Batch 500: Current Loss = 0.3792
Epoch 0, Batch 600: Current Loss = 0.2449
Epoch 0, Batch 700: Current Loss = 0.2986
Epoch 0, Batch 800: Current Loss = 0.4399
Epoch 0, Batch 900: Current Loss = 0.5002
Epoch 0, Test Accuracy: 88.94%
New best accuracy: 88.94%
Best model saved as best_model.pt
Epoch 1, Batch 0: Current Loss = 0.2979
Epoch 1, Batch 100: Current Loss = 0.2503
Epoch 1, Batch 200: Current Loss = 0.2062
Epoch 1, Batch 300: Current Loss = 0.1596
Epoch 1, Batch 400: Current Loss = 0.3097
Epoch 1, Batch 500: Current Loss = 0.3151
Epoch 1, Batch 600: Current Loss = 0.1686
Epoch 1, Batch 700: Current Loss = 0.2702
Epoch 1, Batch 800: Current Loss = 0.3540
Epoch 1, Batch 900: Current Loss = 0.2925
Epoch 1, T

In [11]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
import numpy as np

class ConvolutionalNet(nn.Module):
    def __init__(self):
        super(ConvolutionalNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(1, 32, 3, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(32)
        self.conv_layer2 = nn.Conv2d(32, 64, 3, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(64)
        self.conv_layer3 = nn.Conv2d(64, 64, 3, padding=1)
        self.batch_norm3 = nn.BatchNorm2d(64)
        self.pooling_layer = nn.MaxPool2d(2, 2)
        self.fc_layer1 = nn.Linear(64 * 7 * 7, 512)
        self.dropout_layer = nn.Dropout(0.5)
        self.fc_layer2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pooling_layer(F.relu(self.batch_norm1(self.conv_layer1(x))))
        x = self.pooling_layer(F.relu(self.batch_norm2(self.conv_layer2(x))))
        x = F.relu(self.batch_norm3(self.conv_layer3(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc_layer1(x))
        x = self.dropout_layer(x)
        x = self.fc_layer2(x)
        return x

class ImageDataset(Dataset):
    def __init__(self, path='./data', is_train=True, transform=None):
        self.dataset = datasets.FashionMNIST(
            root=path,
            train=is_train,
            download=True
        )

        self.images = self.dataset.data.numpy()
        self.labels = self.dataset.targets.numpy()
        self.images = self.images.astype(np.float32) / 255.0
        self.transform = transform

        self.class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                            'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        image = torch.FloatTensor(image).unsqueeze(0)

        if self.transform:
            image = self.transform(image)

        return image, label

def save_best_model(model, filepath='best_model.pt'):
    torch.save(model.state_dict(), filepath)
    print(f"Best model saved as {filepath}")

def load_best_model(filepath='best_model.pt'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = ConvolutionalNet().to(device)

    try:
        model.load_state_dict(torch.load(filepath, map_location=device))
        print(f"Model loaded from {filepath}")
        return model
    except FileNotFoundError:
        print(f"Model file {filepath} not found.")
        return model

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Training started on device: {device}")

    train_dataset = ImageDataset(is_train=True)
    test_dataset = ImageDataset(is_train=False)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    model = ConvolutionalNet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_function = nn.CrossEntropyLoss()

    total_epochs = 85
    highest_accuracy = 0.0
    best_model_path = 'best_model.pt'

    print("Training process initiated...")

    for epoch in range(total_epochs):
        model.train()
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            predictions = model(inputs)
            loss = loss_function(predictions, targets)
            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print(f"Epoch {epoch}, Batch {batch_idx}: Current Loss = {loss.item():.4f}")

        model.eval()
        correct_predictions = 0
        total_samples = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                predictions = model(inputs)
                _, predicted_labels = torch.max(predictions.data, 1)
                total_samples += targets.size(0)
                correct_predictions += (predicted_labels == targets).sum().item()

        accuracy = 100 * correct_predictions / total_samples
        print(f"Epoch {epoch}, Test Accuracy: {accuracy:.2f}%")

        if accuracy > highest_accuracy:
            highest_accuracy = accuracy
            print(f"New best accuracy: {accuracy:.2f}%")
            save_best_model(model, best_model_path)

    print(f"\nTraining complete!")
    print(f"Highest accuracy achieved during training: {highest_accuracy:.2f}%")


Training started on device: cuda
Training process initiated...
Epoch 0, Batch 0: Current Loss = 2.3228
Epoch 0, Batch 100: Current Loss = 0.3543
Epoch 0, Batch 200: Current Loss = 0.2278
Epoch 0, Batch 300: Current Loss = 0.5057
Epoch 0, Batch 400: Current Loss = 0.2673
Epoch 0, Batch 500: Current Loss = 0.2710
Epoch 0, Batch 600: Current Loss = 0.3091
Epoch 0, Batch 700: Current Loss = 0.4639
Epoch 0, Batch 800: Current Loss = 0.2305
Epoch 0, Batch 900: Current Loss = 0.2527
Epoch 0, Test Accuracy: 89.63%
New best accuracy: 89.63%
Best model saved as best_model.pt
Epoch 1, Batch 0: Current Loss = 0.2450
Epoch 1, Batch 100: Current Loss = 0.2011
Epoch 1, Batch 200: Current Loss = 0.2248
Epoch 1, Batch 300: Current Loss = 0.3591
Epoch 1, Batch 400: Current Loss = 0.2751
Epoch 1, Batch 500: Current Loss = 0.2265
Epoch 1, Batch 600: Current Loss = 0.2792
Epoch 1, Batch 700: Current Loss = 0.2681
Epoch 1, Batch 800: Current Loss = 0.1370
Epoch 1, Batch 900: Current Loss = 0.1564
Epoch 1, T